In [ ]:
import pandas as pd
import numpy as np
import glob
import os
import shutil

In [ ]:
df_mexico = 'path to the mexico metadata csv file'
csv_folder = 'Path to the folder containing the individual CSV files'
output_folder = 'path to a folder to save the processed CSV files'

In [ ]:
# Loop over each row in the filtered DataFrame
for index, row in df_mexico.iterrows():
    identifier = row['samplingFeatureCode']
    csv_file_path = os.path.join(csv_folder, f'{identifier}.csv')

    # Check if the file exists
    if os.path.exists(csv_file_path):
        # Read the CSV file, skipping the first 7 rows, and setting the next row as header
        df = pd.read_csv(csv_file_path, skiprows=7, header=0)

        # Display current columns for debugging purposes
        print(f"Processing file: {csv_file_path}")
        print(f"Columns before renaming: {df.columns.tolist()}")

        # Rename the first column to 'Datetime'
        df.rename(columns={df.columns[0]: 'Datetime'}, inplace=True)

        # Drop the second column
        df.drop(columns=[df.columns[1]], inplace=True)

        # Rename the third column to 'Streamflow (m3/s)'
        df.rename(columns={df.columns[1]: 'Streamflow (m3/s)'}, inplace=True)

        # Replace '-' with -9999 in 'Streamflow (m3/s)' column
        df['Streamflow (m3/s)'] = df['Streamflow (m3/s)'].replace('-', -9999)

        # Drop rows where 'Streamflow (m3/s)' is NaN or missing
        df.dropna(subset=['Streamflow (m3/s)'], inplace=True)

        # Include only rows between the first and last available data
        # Identify the first and last valid rows (where 'Streamflow (m3/s)' is not -9999)
        valid_rows = df[df['Streamflow (m3/s)'] != -9999]
        if not valid_rows.empty:
            first_index = valid_rows.index[0]
            last_index = valid_rows.index[-1]
            df = df.loc[first_index:last_index]

        # Save the processed file to the output folder
        output_file_path = os.path.join(output_folder, f'{identifier}.csv')
        df.to_csv(output_file_path, index=False)

        print(f'Processed and saved: {output_file_path}')
    else:
        print(f'File not found for identifier: {identifier}')

To clean the data as written in the gauge documentation

In [ ]:
# Function to apply zero-replacement logic
def replace_zeros(streamflow):
    modified_streamflow = streamflow.copy()
    
    # Case 1: Zeros at the start
    if modified_streamflow[0] == 0:
        i = 0
        # Replace all leading zeros with -9999
        while i < len(modified_streamflow) and modified_streamflow[i] == 0:
            modified_streamflow[i] = -9999
            i += 1
    
    # Now, let's check for individual or consecutive zeros in the remaining data
    for i in range(1, len(modified_streamflow)):
        if modified_streamflow[i] == 0:
            # Start of a zero sequence, check the previous value
            prev_value = modified_streamflow[i-1]
            zero_start = i
            
            # Find end of consecutive zero sequence
            while i < len(modified_streamflow) and modified_streamflow[i] == 0:
                i += 1
            zero_end = i  # End of the zero sequence (exclusive)
            
            # If the previous day's value is < 1, keep zeros
            if prev_value < 1:
                continue
            else:
                # Replace all zeros in the sequence with -9999
                modified_streamflow[zero_start:zero_end] = -9999

    return modified_streamflow

# Folder paths
input_folder_path = '/Users/yubin/Library/CloudStorage/Box-Box/Bias Correction/mexico/Hydroserver/Unprocessed'  # Folder where your 1st level processed CSV files are stored
output_folder_path = '/Users/yubin/Library/CloudStorage/Box-Box/Bias Correction/mexico/Hydroserver/Processed'  # Folder where you want to save modified files
summary_file_path = os.path.join('/Users/yubin/Library/CloudStorage/Box-Box/Bias Correction/mexico/Hydroserver/zero_summary.csv')  # Summary CSV file path

# Ensure the output folder exists, if not, create it
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

# List to store summary information
summary_data = []

# Loop through each file in the input folder
for filename in os.listdir(input_folder_path):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_folder_path, filename)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Convert Datetime column if necessary
        df['Datetime'] = pd.to_datetime(df['Datetime'])
        
        # Count the number of zeros before replacement
        initial_zero_count = (df['Streamflow (m3/s)'] == 0).sum()
        
        # Apply the replacement logic
        df['Modified_Streamflow'] = replace_zeros(df['Streamflow (m3/s)'])
        
        # Count the number of zeros after replacement
        final_zero_count = (df['Modified_Streamflow'] == 0).sum()
        
        # Save the modified data back to the output folder
        modified_file_path = os.path.join(output_folder_path, filename.replace(".csv", "_modified.csv"))
        df.to_csv(modified_file_path, index=False)
        
        # Append summary information to the list
        summary_data.append({
            'File': filename,
            'Initial_Zeros': initial_zero_count,
            'Final_Zeros': final_zero_count
        })

# Create a summary DataFrame
summary_df = pd.DataFrame(summary_data)

# Save the summary to a CSV file in the output folder
summary_df.to_csv(summary_file_path, index=False)

print(f"Summary file saved to: {summary_file_path}")

Summary calculation of the mexico data

In [ ]:
# Define the path to the folder containing the CSV files
folder_path = '/Users/yubin/Library/CloudStorage/Box-Box/Bias Correction/mexico/Hydroserver/Processed'
output_path = '/Users/yubin/Library/CloudStorage/Box-Box/Bias Correction/mexico/Hydroserver/'
# Get a list of all CSV files in the folder
csv_files = glob.glob(os.path.join(folder_path, '*.csv'))

# Define the range of years and months
years = list(range(1940, 2024 + 1))
months = list(range(1, 13))

# Create empty DataFrames for yearly and monthly summaries with the required columns
yearly_summary_df = pd.DataFrame(columns=['File'] + years)
monthly_summary_df = pd.DataFrame(columns=['File'] + months)

# Loop over each CSV file to process data
for file in csv_files:
    # Extract the file name without the .csv extension
    file_name = os.path.basename(file).replace('.csv', '')
    
    # Read the CSV file
    data = pd.read_csv(file)
    
    # Convert the 'Datetime' column to datetime type
    data['Datetime'] = pd.to_datetime(data['Datetime'])
    
    # Extract year and month from the 'Datetime' column
    data['Year'] = data['Datetime'].dt.year
    data['Month'] = data['Datetime'].dt.month
    
    # Remove non-numerical values, convert to numeric and remove NaN
    data['Modified_Streamflow'] = pd.to_numeric(data['Modified_Streamflow'], errors='coerce')
    
    # Filter out rows where the value is negative, zero, or NaN
    valid_data = data[(data['Modified_Streamflow'] > 0) & (~data['Modified_Streamflow'].isna())]
    
    # Group by year and count the number of valid (non-negative, non-zero) observations
    yearly_counts = valid_data.groupby('Year').size().reindex(years, fill_value=0).reset_index(name='Valid Observations')
    yearly_counts = yearly_counts.set_index('Year').transpose()
    yearly_counts['File'] = file_name
    yearly_summary_df = pd.concat([yearly_summary_df, yearly_counts], ignore_index=True)
    
    # Group by month and count the number of valid (non-negative, non-zero) observations across all years
    monthly_counts = valid_data.groupby('Month').size().reindex(months, fill_value=0).reset_index(name='Valid Observations')
    monthly_counts = monthly_counts.set_index('Month').transpose()
    monthly_counts['File'] = file_name
    monthly_summary_df = pd.concat([monthly_summary_df, monthly_counts], ignore_index=True)

# Reorder the columns to have 'File' as the first column
yearly_summary_df = yearly_summary_df[['File'] + years]
monthly_summary_df = monthly_summary_df[['File'] + months]

# Define the paths to save the summary CSV files
yearly_summary_file_path = os.path.join(output_path, 'yearly_summary_valid_observations.csv')
monthly_summary_file_path = os.path.join(output_path, 'monthly_summary_valid_observations.csv')

# Save the summary DataFrames to CSV files
yearly_summary_df.to_csv(yearly_summary_file_path, index=False)
monthly_summary_df.to_csv(monthly_summary_file_path, index=False)

print(f"Yearly summary saved to {yearly_summary_file_path}")
print(f"Monthly summary saved to {monthly_summary_file_path}")
